# Trickbot Malware Recon Activity
Trickbot enumerates domain/network topology and executes certain commands automatically every few minutes. This detectors attempts to identify that activity based off a command rarely observed in an enterprise network.

## Rule Content
```
- title: Trickbot Malware Recon Activity
  id: 410ad193-a728-4107-bc79-4419789fcbf8
  status: experimental
  description: Trickbot enumerates domain/network topology and executes certain commands
    automatically every few minutes. This detectors attempts to identify that activity
    based off a command rarely observed in an enterprise network.
  references:
  - https://www.sneakymonkey.net/2019/05/22/trickbot-analysis/
  author: David Burkett
  date: 12/28/2019
  tags:
  - attack.t1482
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image:
      - '*\nltest.exe'
      CommandLine:
      - /domain_trusts /all_trusts
      - /domain_trusts
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Rare System Admin Activity
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:(*\\nltest.exe) AND process_command_line:("\/domain_trusts\ \/all_trusts" OR "\/domain_trusts"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()